In [1]:
import warnings; 
warnings.simplefilter('ignore')

# IMPORT PROPHET AND DATA

In [2]:
import pandas as pd
from fbprophet import Prophet

In [20]:
#pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [4]:
M_df = pd.read_csv('D:/finalProject/Master_data.csv')
M_df.dtypes

state                           object
county                          object
year                             int64
totalpopulation                  int64
unincorporatedpop                int64
incorporatedpop                  int64
average_yearly_wage              int64
lowtiermedianhomevalue         float64
hightiermedianhomevalue        float64
singlefamilymedianhomevalue    float64
value_inches                   float64
murder                         float64
rape                           float64
robbery                        float64
aggravated_assault             float64
burglary                       float64
larceny                        float64
motor_vehicle_theft            float64
index_per_100k                 float64
employment                     float64
unemployment                     int64
labor_force                      int64
unemployment_rate              float64
dtype: object

In [5]:
M_df.county = pd.Categorical(M_df.county)
M_df["county_code"] = M_df.county.cat.codes
M_df.head(10)

,state,county,year,totalpopulation,unincorporatedpop,incorporatedpop,average_yearly_wage,lowtiermedianhomevalue,hightiermedianhomevalue,singlefamilymedianhomevalue,...,aggravated_assault,burglary,larceny,motor_vehicle_theft,index_per_100k,employment,unemployment,labor_force,unemployment_rate,county_code
0,Florida,Alachua,2001,222935,108532,114403,26917,72233.0,198921.0,121316.0,...,1240.0,2918.0,7736.0,1031.0,6083.0,114284.0,4263,118547,0.036,0
1,Florida,Alachua,2002,228607,111939,116668,27648,76757.0,210486.0,129499.0,...,1264.0,2768.0,7429.0,895.0,5637.0,113387.0,4776,118163,0.040,0
2,Florida,Alachua,2003,231296,95161,136135,28867,83740.0,228059.0,141447.0,...,1413.0,2542.0,6782.0,877.0,5225.0,114641.0,4446,119087,0.037,0
3,Florida,Alachua,2004,236174,98755,137419,30939,95977.0,260733.0,162329.0,...,1325.0,2533.0,6064.0,744.0,4733.0,115973.0,4283,120256,0.036,0
4,Florida,Alachua,2005,240764,100507,140257,33145,111818.0,301074.0,187998.0,...,1511.0,2442.0,6448.0,749.0,4852.0,119924.0,3626,123550,0.029,0
5,Florida,Alachua,2006,243779,101950,141829,34748,130667.0,345131.0,217399.0,...,1558.0,2779.0,7530.0,794.0,5454.0,122471.0,3372,125843,0.027,0
6,Florida,Alachua,2007,247561,103217,144344,36456,135485.0,354738.0,224103.0,...,1574.0,2920.0,7070.0,768.0,5229.0,125102.0,3908,129010,0.030,0
7,Florida,Alachua,2008,252388,105051,147337,37469,125463.0,325363.0,206437.0,...,1380.0,2747.0,7179.0,657.0,4950.0,125425.0,5821,131246,0.044,0
8,Florida,Alachua,2009,256232,107260,148972,39080,110449.0,288381.0,183571.0,...,1414.0,2858.0,7253.0,686.0,4988.0,120579.0,9236,129815,0.071,0
9,Florida,Alachua,2010,247336,98991,148345,39688,102223.0,269990.0,172110.0,...,1093.0,2297.0,6387.0,493.0,4198.0,117504.0,9880,127384,0.078,0


In [32]:
# Check Null values before fitting features into Prophet.
for column in M_df.columns:
    print(f"Column {column} has {M_df[column].isnull().sum()} null values")

Column state has 0 null values
Column county has 0 null values
Column year has 0 null values
Column totalpopulation has 0 null values
Column unincorporatedpop has 0 null values
Column incorporatedpop has 0 null values
Column average_yearly_wage has 0 null values
Column lowtiermedianhomevalue has 36 null values
Column hightiermedianhomevalue has 36 null values
Column singlefamilymedianhomevalue has 36 null values
Column value_inches has 0 null values
Column murder has 137 null values
Column rape has 137 null values
Column robbery has 137 null values
Column aggravated_assault has 137 null values
Column burglary has 137 null values
Column larceny has 137 null values
Column motor_vehicle_theft has 137 null values
Column index_per_100k has 139 null values
Column employment has 1 null values
Column unemployment has 0 null values
Column labor_force has 0 null values
Column unemployment_rate has 0 null values
Column county_code has 0 null values


In [6]:
M_df.shape

(1340, 24)

# CONVERT YEAR TO DT FOR PROPHET MODEL

In [7]:
M_df['year'] = M_df['year'].map(str) + '-12-31'
M_df['year'] = pd.DatetimeIndex(M_df['year'])
M_df['year'] = M_df['year'].dt.date
M_df['year']

0       2001-12-31
1       2002-12-31
2       2003-12-31
3       2004-12-31
4       2005-12-31
           ...    
1335    2016-12-31
1336    2017-12-31
1337    2018-12-31
1338    2019-12-31
1339    2020-12-31
Name: year, Length: 1340, dtype: object

# TOTAL POPULATION FORECAST

In [8]:
population_df = pd.DataFrame(columns=['ds','yhat'])

In [10]:
for i in range(len(M_df.county_code.unique())):
    t_df = M_df.loc[M_df['county_code']==i]
    t_df = t_df.loc[:,["year","totalpopulation"]]
    t_df.columns = ['ds','y']
    
    m = Prophet(yearly_seasonality=True)
    model = m.fit(t_df)
    
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    
    population_df = pd.concat([population_df, forecast[['ds','yhat']]], axis=0,ignore_index=True)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

In [12]:
population_df

,ds,yhat
0,2001-12-31,223752.337576
1,2002-12-31,227557.808861
2,2003-12-31,231732.066172
3,2004-12-31,236653.464818
4,2005-12-31,240100.186603
...,...,...
2005,2026-12-31,25690.524995
2006,2027-12-31,25722.888969
2007,2028-12-31,25938.552678
2008,2029-12-31,25949.108443


In [51]:
file_path = "D:/finalProject/MLtest/Population_Forecast.csv"
population_df.to_csv(file_path, index=False)

# LABOR FORCE FORECAST

In [15]:
labor_force_df = pd.DataFrame(columns=['ds','yhat'])

In [27]:
for i in range(len(M_df.county_code.unique())):
    lf_df = M_df.loc[M_df['county_code']==i]
    lf_df = lf_df.loc[:,["year","labor_force"]]
    lf_df.columns = ['ds','y']
    
    m = Prophet(yearly_seasonality=True)
    model = m.fit(lf_df)
    
    future2 = m.make_future_dataframe(periods=10,freq='Y')
    forecast_lf = m.predict(future2)
    
    labor_force_df = pd.concat([labor_force_df, forecast_lf[['ds','yhat']]], axis=0,ignore_index=True)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

In [28]:
labor_force_df

,ds,yhat
0,2001-12-31,1.173611e+05
1,2002-12-31,1.190110e+05
2,2003-12-31,1.204236e+05
3,2004-12-31,1.219670e+05
4,2005-12-31,1.238549e+05
5,2006-12-31,1.255048e+05
6,2007-12-31,1.269170e+05
7,2008-12-31,1.274055e+05
8,2009-12-31,1.282334e+05
9,2010-12-31,1.288224e+05


In [52]:
file_path = "D:/finalProject/MLtest/LaborForce_Forecast.csv"
labor_force_df.to_csv(file_path, index=False)

# AVERAGE YEARLY WAGE FORECAST

In [22]:
avg_wage_df = pd.DataFrame(columns=['ds','yhat'])

In [23]:
for i in range(len(M_df.county_code.unique())):
    ayw_df = M_df.loc[M_df['county_code']==i]
    ayw_df = ayw_df.loc[:,["year","average_yearly_wage"]]
    ayw_df.columns = ['ds','y']
    
    m = Prophet(yearly_seasonality=True)
    model = m.fit(ayw_df)
    
    future3 = m.make_future_dataframe(periods=10,freq='Y')
    forecast_ayw = m.predict(future3)
    
    avg_wage_df = pd.concat([avg_wage_df, forecast_ayw[['ds','yhat']]], axis=0,ignore_index=True)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

In [24]:
avg_wage_df

,ds,yhat
0,2001-12-31,26438.784122
1,2002-12-31,27961.417732
2,2003-12-31,29354.450203
3,2004-12-31,31391.501617
4,2005-12-31,33043.912684
5,2006-12-31,34566.546298
6,2007-12-31,35958.501496
7,2008-12-31,37738.400493
8,2009-12-31,38587.162919
9,2010-12-31,39301.354697


In [53]:
file_path = "D:/finalProject/MLtest/AvgYrWage_Forecast.csv"
avg_wage_df.to_csv(file_path, index=False)

# EMPLOYMENT FORECAST

In [47]:
# Take rows that do not have null values in feature column.
M2_df = M_df[M_df['employment'].notna()]
M2_df.shape

(1339, 24)

In [36]:
emp_df = pd.DataFrame(columns=['ds','yhat'])

In [37]:
for i in range(len(M2_df.county_code.unique())):
    e_df = M2_df.loc[M2_df['county_code']==i]
    e_df = e_df.loc[:,["year","employment"]]
    e_df.columns = ['ds','y']
    
    m = Prophet(yearly_seasonality=True)
    model = m.fit(e_df)
    
    future4 = m.make_future_dataframe(periods=10,freq='Y')
    forecast_e = m.predict(future4)
    
    emp_df = pd.concat([emp_df, forecast_e[['ds','yhat']]], axis=0,ignore_index=True)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 14.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

In [38]:
emp_df

,ds,yhat
0,2001-12-31,1.140838e+05
1,2002-12-31,1.155012e+05
2,2003-12-31,1.175110e+05
3,2004-12-31,1.166330e+05
4,2005-12-31,1.174577e+05
5,2006-12-31,1.188752e+05
6,2007-12-31,1.208850e+05
7,2008-12-31,1.200069e+05
8,2009-12-31,1.208317e+05
9,2010-12-31,1.222491e+05


In [54]:
file_path = "D:/finalProject/MLtest/Employment_Forecast.csv"
emp_df.to_csv(file_path, index=False)

## SINGLE FAMILY HOME MEDIAN VALUE FORECAST

In [46]:
# Take rows that do not have null values in feature column.
M3_df = M_df[M_df['singlefamilymedianhomevalue'].notna()]
M3_df.shape

(1303, 24)

In [48]:
singlehome_df = pd.DataFrame(columns=['ds','yhat'])

In [49]:
for i in range(len(M3_df.county_code.unique())):
    sfmhv_df = M3_df.loc[M3_df['county_code']==i]
    sfmhv_df = sfmhv_df.loc[:,["year","singlefamilymedianhomevalue"]]
    sfmhv_df.columns = ['ds','y']
    
    m = Prophet(yearly_seasonality=True)
    model = m.fit(sfmhv_df)
    
    future5 = m.make_future_dataframe(periods=10,freq='Y')
    forecast_sfmhv = m.predict(future5)
    
    singlehome_df = pd.concat([singlehome_df, forecast_sfmhv[['ds','yhat']]], axis=0,ignore_index=True)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 14.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 13.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

In [50]:
singlehome_df

,ds,yhat
0,2001-12-31,154270.297922
1,2002-12-31,158552.040315
2,2003-12-31,163629.152477
3,2004-12-31,159622.482489
4,2005-12-31,163106.276322
5,2006-12-31,167388.018714
6,2007-12-31,172465.130876
7,2008-12-31,168458.460885
8,2009-12-31,171942.254712
9,2010-12-31,176223.997098


In [55]:
file_path = "D:/finalProject/MLtest/SingleFamHomeMed_Forecast.csv"
singlehome_df.to_csv(file_path, index=False)